In [1]:
from speckle import VirtExp
import speckle
import numpy as np
import math
import bpy
import mathutils

In [3]:
pattern_path = r"D:\Experiment Quality\blender_model\im.tiff"
normals_path = r"D:\Experiment Quality\blender_model\grad.tiff"
model_path = r"D:\Experiment Quality\blender_model\model_dev.blend"
output_path = r"D:\Experiment Quality\Correlation test\compliant_spec_distortion\render_0_0.tiff"
output_path2 = r"D:\Experiment Quality\Correlation test\compliant_spec_distortion\render_0_1.tiff"
calib_path = r"D:\Experiment Quality\Correlation test\compliant_spec_distortion\calibration.caldat"
mesh_path = r"D:\GitHub\speckle\test_specimen\fullSpec.mesh"

In [11]:
# Generate blender scene
a = VirtExp(pattern_path, normals_path, output_path, model_path,
            objects_position="fixed")
# Set up the scene
# Get default properties
p = a.get_default_params()
# CAMERAS
# Add the default target
target = a.add_FEA_part(mesh_path)
# Add the light panel
# Calculate desired orientation of the light
light_target_orient = p["light_target"] - np.array(p["light_pos"])
# Calculate the rotation angle of the light
light_angle = a.calc_rot_angle(p["light_init_rot"], 
                                  light_target_orient)
a.add_light(p["light_type"], pos=p["light_pos"], orient=light_angle,
               energy=p["light_energy"], spot_size=p["light_spotsize"],
               spot_blend=p["light_spot_blend"], 
               shadow_spot_size = p["light_shad_spot"])
# Add straight camera
# Calculate desired orientation of the cam
cam0_target_orient = p["cam0_target"] - np.array(p["cam0_pos"]) 
cam0_target_dist = np.linalg.norm(cam0_target_orient)+1e-16
cam_angle = a.calc_rot_angle(p["cam_init_rot"], cam0_target_orient)
cam0 = a.add_camera(pos=p["cam0_pos"], orient=cam_angle, 
                       fstop=p["cam_fstop"], 
                       focal_length=p["cam_foc_length"],
                       obj_distance=cam0_target_dist,
                       k1=0.0, p1=0.00)  
# Add cross camera
# Camera position
#ang_x = math.radians(20.0)
#cam1_pos = np.array([0.0, math.sin(ang_x), math.cos(ang_x)])
#ang_y = math.radians(15.0)
#cam1_pos = np.array([math.sin(ang_y), 0, math.cos(ang_y)])
cam1_pos = np.array([0.15, 0.02, 0.99])
cam1_target_orient = p["cam1_target"] - cam1_pos
cam1_target_dist = np.linalg.norm(cam1_target_orient)+1e-16
cam_angle = a.calc_rot_angle(p["cam_init_rot"], cam1_target_orient)
cam1 = a.add_camera(pos=cam1_pos, orient=cam_angle, 
                       fstop=p["cam_fstop"],
                       focal_length=p["cam_foc_length"],
                       obj_distance=cam1_target_dist)
a.rotate_around_z(cam1, -1.5)
# Define the material and assign it to the cube
a.add_material(target)
# Write the calibration file
a.generate_calib_file(cam0, cam1, calib_path, ang_mode='XYZ')
# Set the renderer up and render image
a.set_renderer(cam0, n_samples=500)
# Add distortion to the model
a.add_image_distortion(cam0)
# Save the model
a.save_model()
# Render the scene with the perpendicular camera
a.render_scene(output_path)
# Switch the camera to the cross one and render the scene
a.set_renderer(cam1, n_samples=500)
# Add distortion to the model
a.add_image_distortion(cam1)
a.render_scene(output_path2) 

Info: Saved "model_dev.blend"


In [12]:
a.generate_calib_file(cam0, cam1, calib_path, ang_mode='ZXY')

# Check how translation changes over rotation

In [20]:
T = (cam0.location - cam1.location) * 1000
print(T)
R = np.array([[1, 0, 0], [0, math.cos(ang_x), math.sin(ang_x)], \
             [0, -math.sin(ang_x), math.cos(ang_x)]])
print(R)
R.transpose() @ T

<Vector (0.0000, -342.0201, 60.3074)>
[[ 1.          0.          0.        ]
 [ 0.          0.93969262  0.34202014]
 [ 0.         -0.34202014  0.93969262]]


array([   0.        , -342.02014351,  -60.30737367])

In [23]:
print(cam1_target_orient)


[ 0.42261826  0.         -0.90630779]


In [33]:
def quaternion_multiply(quaternion1, quaternion0):
    w0, x0, y0, z0 = quaternion0
    w1, x1, y1, z1 = quaternion1
    return np.array([-x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0,
                     x1 * w0 + y1 * z0 - z1 * y0 + w1 * x0,
                     -x1 * z0 + y1 * w0 + z1 * x0 + w1 * y0,
                     x1 * y0 - y1 * x0 + z1 * w0 + w1 * z0], dtype=np.float64)

In [46]:
ang_y = math.radians(15.0)
cam1_pos = np.array([math.sin(ang_y), 0, math.cos(ang_y)])
cam1_target_orient = p["cam1_target"] - cam1_pos
v1 = p["cam_init_rot"]
v2 = cam1_target_orient
a = np.cross(v1, v2);
q = a;
q = np.insert(q, 0, np.sqrt((np.linalg.norm(v1) ** 2) * (np.linalg.norm(v2) ** 2)) + np.dot(v1, v2))
q /= np.linalg.norm(q)
print(q)
ang_z = math.radians(5)
z_axis = np.array([0, 0, 1])
qz = np.array([math.cos(ang_z/2), 
               z_axis[0]*math.sin(ang_z/2), 
               z_axis[1]*math.sin(ang_z/2), 
               z_axis[2]*math.sin(ang_z/2)])
qz /= np.linalg.norm(qz)
print(qz)
q2 = quaternion_multiply(q, qz)
print(q2)
# Try to rotate first around y and then z
Ry = np.array([[math.cos(ang_y), 0, math.sin(ang_y)], 
               [0, 1, 0],
               [-math.sin(ang_y), 0, math.cos(ang_y)]])
z_axis_rot = Ry @ z_axis
print('Rotating around y first, then z')
print(z_axis_rot)
qz = np.array([math.cos(ang_z/2), 
               z_axis_rot[0]*math.sin(ang_z/2), 
               z_axis_rot[1]*math.sin(ang_z/2), 
               z_axis_rot[2]*math.sin(ang_z/2)])
qz /= np.linalg.norm(qz)
print(qz)
q2 = quaternion_multiply(qz, q)
print(q2)

[0.99144486 0.         0.13052619 0.        ]
[0.99904822 0.         0.         0.04361939]
[0.99050123 0.00569347 0.13040196 0.04324622]
Rotating around y first, then z
[0.25881905 0.         0.96592583]
[0.99904822 0.01128953 0.         0.04213309]
[0.99050123 0.00569347 0.13040196 0.04324622]


# Calculation of the rotation angle between the two cameras

The relative orientation between two quaternions can be found as:

$q = q_{0} * inverse(q_{1})$

Where:

$inverse(q_{1})$ is its conjugate: $q_{1}^{*} = [q, -x, -y, -z]$

In [82]:
q_cam0 = [0, 0, 0, -1]
# q_cam1 = [0.99050123, 0.00569347, 0.13040196, 0.04324622] # with 5deg around Z
#q_cam1 = [0.991445, 0, 0.130526, 0] # without rot around z
q_cam1 = [0, -0.2588, 0, -0.965925]
q_cam0_conj = [-j if i > 0 else j for i,j in enumerate(q_cam0)]
q_cam1_conj = [-j if i > 0 else j for i,j in enumerate(q_cam1)]

rot = quaternion_multiply(q_cam1, q_cam0_conj)
rot_conj = [-j if i > 0 else j for i,j in enumerate(rot)]
print(rot)

[0.965925 0.       0.2588   0.      ]


## Convert the rotation quaternion into the Euler angles using bpy functions

In [84]:
q_rot = mathutils.Quaternion(rot)
print(q_rot)
q_rot_eul = q_rot.to_euler('XYZ')
q_rot_eul = [math.degrees(i) for i in q_rot_eul]
print(q_rot_eul)

<Quaternion (w=0.9659, x=0.0000, y=0.2588, z=0.0000)>
[0.0, 29.997924457318373, 0.0]


## Rotate the translation vector from camera 0 to camera 1

In [78]:
T = [0.258819, 0, 1-0.96593]
T.insert(0,0)
q_T = np.array(T)
print(T)
q_T_rot = quaternion_multiply(quaternion_multiply(rot, q_T), rot_conj)
print(q_T_rot)


[0, 0.258819, 0, 0.034070000000000045]
[ 0.          0.25881799  0.         -0.03407809]
